In [114]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import pickle
import os
from  pdfid_v0_2_8 import pdfid
from  pdf_parser_V0_7_8 import pdf_parser
from  xml.dom import minidom
import xmltodict
import json
import subprocess


In [115]:
#Load dataset as pandas dataframe
df = pd.read_csv("Dataset/PDFMalware.csv")
#Get column names
print(df.columns)
#df.info()


Index(['Fine name', 'pdfsize', 'metadata size', 'pages', 'xref Length',
       'title characters', 'isEncrypted', 'embedded files', 'images', 'text',
       'header', 'obj', 'endobj', 'stream', 'endstream', 'xref', 'trailer',
       'startxref', 'pageno', 'encrypt', 'ObjStm', 'JS', 'Javascript', 'AA',
       'OpenAction', 'Acroform', 'JBIG2Decode', 'RichMedia', 'launch',
       'EmbeddedFile', 'XFA', 'Colors', 'Class'],
      dtype='object')


In [116]:

df_features = df[["encrypt","xref","obj","ObjStm","startxref","JBIG2Decode","Acroform","stream","Javascript","JS","OpenAction","RichMedia","launch","EmbeddedFile","XFA","AA"]]
label_encoder = preprocessing.LabelEncoder() 


feature_list = ["xref","obj","ObjStm","startxref","JBIG2Decode","Acroform","Javascript","JS","OpenAction","RichMedia","launch","EmbeddedFile","XFA","AA"]

encoder_list = []
for feature_name in feature_list:
    test_feature_name = feature_name + "_coded"
    encoder = label_encoder.fit(df[feature_name]) 
    with open("Models/" + test_feature_name,'wb') as f:
        pickle.dump(encoder,f)
    df_features[test_feature_name] = label_encoder.transform(df[feature_name]) 

df_features_trans = df_features[['encrypt','stream','obj_coded','ObjStm_coded','startxref_coded','JBIG2Decode_coded',"Acroform_coded",'Javascript_coded', 'JS_coded','OpenAction_coded','RichMedia_coded','launch_coded','EmbeddedFile_coded','XFA_coded','AA_coded']]
#df_features_trans.info()


C:\Users\91743\AppData\Local\Temp\ipykernel_147404\358580565.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features[test_feature_name] = label_encoder.transform(df[feature_name])
C:\Users\91743\AppData\Local\Temp\ipykernel_147404\358580565.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features[test_feature_name] = label_encoder.transform(df[feature_name])
C:\Users\91743\AppData\Local\Temp\ipykernel_147404\358580565.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [117]:
df_result = df["Class"]
s = pd.Series(df_result, dtype="category")
df_result_trans = s.cat.codes
df_result_trans.value_counts()


 1    5557
 0    4468
-1       1
dtype: int64

In [118]:
feature_train, feature_test, result_train, result_test = train_test_split(df_features_trans, df_result_trans, test_size=0.20, random_state=101)
print(f'feature_train : {feature_train.shape}')
print(f'feature_test : {feature_test.shape}')
print(f'result_train : {result_train.shape}')
print(f'result_test : {result_test.shape}')

feature_train : (8020, 15)
feature_test : (2006, 15)
result_train : (8020,)
result_test : (2006,)


In [119]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['log2', 'sqrt', None]
# Maximum number of levels in tree
max_depth = [5,8,11,14]
# Minimum number of samples required to split a node
min_samples_split = [2,3,4,5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [False]

param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(param_grid)

{'n_estimators': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200], 'max_features': ['log2', 'sqrt', None], 'max_depth': [5, 8, 11, 14], 'min_samples_split': [2, 3, 4, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [False]}


In [120]:
rfc_Model = RandomForestClassifier()
rfc_Grid = GridSearchCV(estimator = rfc_Model, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)
rfc_Grid.fit(feature_train , result_train )
rfc_Grid.best_params_


C:\Users\91743\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Fitting 3 folds for each of 960 candidates, totalling 2880 fits


In [ ]:
rfc_Grid.score(feature_train,result_train)

0.9861596009975062

In [ ]:
rfc_Grid.score(feature_test,result_test)

0.9765702891326022

In [ ]:


classifier= RandomForestClassifier(bootstrap= rfc_Grid.best_params_["bootstrap"],
 max_depth= rfc_Grid.best_params_["max_depth"],
 max_features= rfc_Grid.best_params_["max_features"],
 min_samples_leaf= rfc_Grid.best_params_["min_samples_leaf"],
 min_samples_split= rfc_Grid.best_params_["min_samples_split"],
 n_estimators= rfc_Grid.best_params_["n_estimators"])  
pdf_checker_RT = classifier.fit(feature_train,result_train)  


In [ ]:
with open("Models/pdf_checker_RT",'wb') as f:
    pickle.dump(pdf_checker_RT,f)

In [ ]:


pdf_file = "Class10.pdf"
result = pdfid.PDFiD(pdf_file)
data_dict = pdfid.PDFiD2JSON(result,True)
json_dict = json.loads(data_dict)

command = f'python ./pdf_parser_V0_7_8/pdf_parser.py -k /URI -O ./{pdf_file}'

p = subprocess.check_output(command, stderr=subprocess.PIPE, shell=True)

uri_list = p.decode().split("\n")
print(uri_list)
if(uri_list[0] == ''):
    uri_number = 0
else:
    uri_number = len(uri_list)
print(uri_number)   




data_list = json_dict[0]["pdfid"]["keywords"]["keyword"]
data_dict_master = {}
for data in data_list:
    data_dict = {data["name"].replace("/",""):[data["count"]]}
    data_dict_master = data_dict_master | data_dict
print(data_dict_master)

df = pd.DataFrame.from_dict(data_dict_master)

df_input = df[["Encrypt","stream","obj","ObjStm","startxref","JBIG2Decode","AcroForm","JavaScript","JS","OpenAction","RichMedia","Launch","EmbeddedFile","XFA","AA"]]

df_input.rename(columns = {'Encrypt':'encrypt','JavaScript':'Javascript','AcroForm':'Acroform','Launch':'launch'}, inplace = True)

feature_list = ["xref","obj","ObjStm","startxref","JBIG2Decode","Acroform","Javascript","JS","OpenAction","RichMedia","launch","EmbeddedFile","XFA","AA"]


for feature_name in feature_list:
    test_feature_name = feature_name + "_coded"
   
    with open("Models/" + test_feature_name,'rb') as f:
        model = pickle.load(f)
    try:
        df_input[test_feature_name] = model.transform( [str(df_input[feature_name][0])] )
    except Exception as e:
        list(model.classes_)
        #print(e)
        continue

df_input_predict = df_input[['encrypt','stream','obj_coded','ObjStm_coded','startxref_coded','JBIG2Decode_coded','Acroform_coded','Javascript_coded', 'JS_coded','OpenAction_coded','RichMedia_coded','launch_coded','EmbeddedFile_coded','XFA_coded','AA_coded']]

['']
0
{'obj': [7], 'endobj': [7], 'stream': [2], 'endstream': [2], 'xref': [1], 'trailer': [1], 'startxref': [1], 'Page': [1], 'Encrypt': [0], 'ObjStm': [0], 'JS': [0], 'JavaScript': [0], 'AA': [0], 'OpenAction': [0], 'AcroForm': [0], 'JBIG2Decode': [0], 'RichMedia': [0], 'Launch': [0], 'EmbeddedFile': [0], 'XFA': [0], 'Colors > 2^24': [0]}


C:\Users\91743\AppData\Local\Temp\ipykernel_147404\632431768.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter.rename(columns = {'Encrypt':'encrypt','JavaScript':'Javascript','AcroForm':'Acroform','Launch':'launch'}, inplace = True)
C:\Users\91743\AppData\Local\Temp\ipykernel_147404\632431768.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[test_feature_name] = model.transform( [str(df_filter[feature_name][0])] )
C:\Users\91743\AppData\Local\Temp\ipykernel_147404\632431768.py:52: SettingWithCopyWarning: 
A value is trying to be set on a

In [ ]:
with open("Models/pdf_checker_RT",'rb') as f:
    model = pickle.load(f)
result = model.predict(df_input_predict)
print(result)

[0]
